<a href="https://colab.research.google.com/github/doudi25/AI_teacher/blob/main/Ai_teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

# Step 2: Import the necessary libraries
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to('cuda')

In [21]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
import aiohttp
import asyncio
import io
import PIL
import google.generativeai as genai
genai.configure = getattr(genai.configure, "func", genai.configure)

model1 = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
)
chat_session = model1.start_chat(history=["do not output asterisk"])
def answer_gen(chat_session,transcription):
  answer = chat_session.send_message(transcription)
  return answer.text

In [3]:
# 1️⃣ Install dependencies silently
!git clone https://huggingface.co/hexgrad/Kokoro-82M
%cd Kokoro-82M
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
!pip install -q phonemizer torch transformers scipy munch


fatal: destination path 'Kokoro-82M' already exists and is not an empty directory.
/content/Kokoro-82M


In [4]:


# 2️⃣ Build the model and load the default voicepack
from models import build_model
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL = build_model('kokoro-v0_19.pth', device)
VOICE_NAME = [
    'af', # Default voice is a 50-50 mix of Bella & Sarah
    'af_bella', 'af_sarah', 'am_adam', 'am_michael',
    'bf_emma', 'bf_isabella', 'bm_george', 'bm_lewis',
    'af_nicole', 'af_sky',
][0]
VOICEPACK = torch.load(f'voices/{VOICE_NAME}.pt', weights_only=True).to(device)
print(f'Loaded voice: {VOICE_NAME}')

from kokoro import generate
text = "How could I know? It's an unanswerable question. Like asking an unborn child if they'll lead a good life. They haven't even been born."
audio, out_ps = generate(MODEL, text, VOICEPACK, lang=VOICE_NAME[0])




/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Loaded voice: af


hˌaʊ kʊd aɪ nˈoʊ? ɪts ɐn ʌnˈænsɚɹəbəl kwˈɛstʃən. lˈaɪk ˈæskɪŋ ɐn ʌnbˈɔːɹn tʃˈaɪld ɪf ðeɪl lˈiːd ɐ ɡˈʊd lˈaɪf. ðeɪ hˈævənt ˈiːvən bˌɪn bˈɔːɹn.


In [22]:
def get_response(audio_path):
  audio_input, sample_rate = librosa.load(audio_path, sr=16000)

  input_features = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_features.to('cuda')

  predicted_ids = model.generate(input_features, forced_decoder_ids=processor.get_decoder_prompt_ids(language="en", task="transcribe"))

  transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
  text_response = answer_gen(chat_session,transcription[0])
  audio,_ = generate(MODEL, text_response, VOICEPACK, lang=VOICE_NAME[0])
  display(Audio(data=audio, rate=24000, autoplay=True))
  return f"Transcription: {transcription}\nResponse: {text_response}", (24000, audio)


In [6]:

!pip install gradio

In [24]:
import gradio as gr
interface = gr.Interface(
    fn=get_response,
    inputs=gr.Audio(sources="microphone", type="filepath", label="Speak something"),
    outputs=[
        gr.Textbox(label="Transcription and Response"),
        gr.Audio(label="Generated Speech")
    ],
    title="Speech-to-Speech AI Assistant",
    description="Speak into the microphone, and the AI will transcribe, respond, and return the audio reply."
)

# Launch the Gradio app
interface.launch(True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://781ffcf3d8491237cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
